In [16]:
import numpy as np
import pandas as pd
import re
from markdown import markdown
from bs4 import BeautifulSoup

train_df = pd.read_csv('../dataset/train.csv', usecols=['readme', 'description'])
val_df = pd.read_csv('../dataset/validation.csv', usecols=['readme', 'description'])
test_df = pd.read_csv('../dataset/test.csv', usecols=['readme', 'description'])

In [17]:
"""
    Return item and drop from frame. Raise KeyError if not found.
"""
def pop(df : pd.DataFrame, idx : int):
    readme = df['readme'][idx]
    description = df['description'][idx]
    result = {'readme' : readme, 'description' : description}
    df.at[idx, 'readme'] = np.nan
    df.at[idx, 'description'] = np.nan
    return result

# Few-shots prompting
def generate_testing_prompt(readme, shots):
    if len(shots) == 0:
        return f"""### Instruction: Summarize the following README contents with LESS THAN 30 words. Your answer should be based on the provided README contents only.

        ### README contents:
        {readme.strip()}

        ### Summary:
        """.strip()
    else:
        prompt = """### Instruction: Summarize the following README contents with LESS THAN 30 words. Your answer should be based on the provided README contents only.
        ### For examples:
        """
        
        for i in range(len(shots)):
            prompt += f""" 
            ### README contents: 
            {shots[i]['readme'].strip()}
            
            ### Summary:
            {shots[i]['description'].strip()}            
            """

        prompt += f"""
        ### README contents:
        {readme.strip()}

        ### Summary:
        """.strip()
    return prompt
        
# Function to remove tags
def format_entry(md_data) :
    html = markdown(md_data)
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
    for a in soup.findAll('a', href=True):
        a.decompose()
    for data in soup(['style', 'script', 'img', 'pre', 'code']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"#+", " ", text)
    return re.sub(r"\^[^ ]+", "", text)

def process_description(s: str) -> str:
    if s.endswith('.'):
        s = s[:-1]
        s = re.sub(r"\. ", ", ", s)
    return s + '.'

def generate_testing_prompt(readme, shots):
    if len(shots) == 0:
        return f"""### Instruction: Summarize the following README contents with LESS THAN 30 words. Your answer should be based on the provided README contents only.

        ### README contents:
        {readme.strip()}

        ### Summary:
        """.strip()
    else:
        prompt = """### Instruction: Summarize the following README contents with LESS THAN 30 words. Your answer should be based on the provided README contents only.
        ### For examples:
        """
        
        for i in range(len(shots)):
            prompt += f""" 
            ### README contents: 
            {shots[i]['readme'].strip()}
            
            ### Summary:
            {shots[i]['description'].strip()}            
            """

        prompt += f"""
        ### README contents:
        {readme.strip()}

        ### Summary:
        """.strip()
        return prompt

In [18]:
for i, readme in enumerate(test_df['readme']):
    test_df.at[i, 'readme'] = format_entry(readme)

In [19]:
shots = []
shots.append(pop(test_df, 8))
shots.append(pop(test_df, 10))
# shots.append(pop(test_df, 42))
# shots.append(pop(test_df, 44))
# shots.append(pop(test_df, 56))

In [20]:
readme = test_df['readme'][0]
generate_testing_prompt(readme, shots)

'### Instruction: Summarize the following README contents with LESS THAN 30 words. Your answer should be based on the provided README contents only.\n        ### For examples:\n         \n            ### README contents: \n            b\'NUI\\n===\\n\\n\\n\\n\\n\\n\\nStyle iOS apps with a stylesheet, similar to CSS\\n\\nDescription\\n-----------\\nNUI is a drop-in UI kit for iOS that lets you style UI elements using a stylesheet, similar to CSS. It lets you style an entire app in minutes.\\n\\nOut of the box, the styling looks like this:\\n\\n\\n\\nIts easily modified, though. The styling above, for example, is declared . Here are examples of other themes that are defined :\\n\\n\\n\\n\\n\\n\\nThe styling is declared using a CSS-like syntax that supports variables:\\n\\n\\n\\nNUI lets you:\\n\\n Update the visual appearance of an entire application in minutes\\n Save themes for use in multiple applications\\n Set the styles of UI elements using simple rules, like setting \\n Define var